In [ ]:
from pathlib import Path

import xarray as xr
import numpy as np
import holoviews as hv
from holoviews.operation import Operation
from holoviews.operation.datashader import rasterize

hv.extension("bokeh")

In [ ]:
opts = {
    "fontscale": 4,
    "xlim": (-30, -10),
    "ylim": (10, 30),
    "frame_height": 800,
    "frame_width": 800,
    "active_tools": [],
    "tools": ["hover"],
}
hv.opts.defaults(
    hv.opts.Image(**opts),
    hv.opts.RGB(**opts)
)

In [ ]:
class rgb(Operation):
    # https://holoviews.org/user_guide/Transforming_Elements.html#defining-custom-operations
    def _process(self, overlay, key=None):
        dataset = xr.merge((image.data for image in overlay.values()))
        array = dataset.to_array("band").transpose(..., "band")
        return hv.RGB(array)

In [ ]:
granule = Path("data/granules")
layout_view = []
for i, item in enumerate(sorted(granule.glob("VNP02MOD.*.nc"))):
    stem = item.stem.split(".")
    granule_id = ".".join(stem[1:3])
    level1 = xr.open_dataset(item, group="observation_data", mask_and_scale=True, chunks={})
    if "M03" in level1:
        path = next(granule.glob(f"VNP03MOD.{granule_id}.*"))
        geoloc = xr.open_dataset(path, group="geolocation_data", chunks={})
        level1 = level1.assign_coords(geoloc[["longitude", "latitude"]])
        level1 = level1[["M05", "M04", "M03"]] / np.cos(np.deg2rad(geoloc["solar_zenith"]))
        # clip to [0, 1] for true color plot
        level1 = level1.where(level1 <= 1)
        # gamma adjustment for true color plot
        gamma = 2.2
        level1 = level1 ** (1/gamma)
        red, green, blue = [rasterize(hv.QuadMesh(level1[i])) for i in ("M05", "M04", "M03")]
        layout_view.append(
            rgb(red * green * blue)
        )
    else:
        path = next(granule.glob(f"VNP02DNB.{granule_id}.*"))
        dnb = xr.open_dataset(path, group="observation_data", mask_and_scale=True, chunks={})
        path = next(granule.glob(f"VNP03DNB.{granule_id}.*"))
        geoloc = xr.open_dataset(path, group="geolocation_data", chunks={})
        dnb = dnb.assign_coords(geoloc[["longitude", "latitude"]])
        dnb = dnb["DNB_observations"]
        dnb = np.log10(dnb.where(dnb > 0))
        dnb["longitude"].attrs.clear()
        dnb["latitude"].attrs.clear()
        layout_view.append(
            rasterize(hv.QuadMesh(dnb))
            .redim.range(DNB_observations=(-12.1, -6.9))
            .opts(cmap="Inferno", colorbar=True, clabel="log(DNB)")
        )

In [ ]:
hv.Layout(layout_view).cols(1)

In [ ]:
granule = Path("data/granules")
prediction = Path("data/granules-prediction-cm")
layout_pred_cm = []
for i, item in enumerate(sorted(prediction.glob("VNP02MOD.*.nc"))):
    stem = item.stem.split(".")
    granule_id = ".".join(stem[1:3])
    ds = xr.open_dataset(item)
    path = next(granule.glob(f"VNP03MOD.{granule_id}.*"))
    geoloc = xr.open_dataset(path, group="geolocation_data")
    ds = ds.assign_coords(geoloc[["longitude", "latitude"]])
    ds["longitude"].attrs.clear()
    ds["latitude"].attrs.clear()
    layout_pred_cm.append(
        rasterize(hv.QuadMesh(np.power(10, ds["y_pred"])))
        .redim.range(y_pred=(-0.1, 4.1))
        .opts(cmap="bjy", colorbar=True, clabel="AOT at 10um")
    )

In [ ]:
hv.Layout(layout_pred_cm).cols(1)

In [ ]:
granule = Path("data/granules")
prediction = Path("data/granules-prediction")
layout_pred = []
for i, item in enumerate(sorted(prediction.glob("VNP02MOD.*.nc"))):
    stem = item.stem.split(".")
    granule_id = ".".join(stem[1:3])
    ds = xr.open_dataset(item)
    path = next(granule.glob(f"VNP03MOD.{granule_id}.*"))
    geoloc = xr.open_dataset(path, group="geolocation_data")
    ds = ds.assign_coords(geoloc[["longitude", "latitude"]])
    ds["longitude"].attrs.clear()
    ds["latitude"].attrs.clear()
    layout_pred.append(
        rasterize(hv.QuadMesh(np.power(10, ds["y_pred"])))
        .redim.range(y_pred=(-0.1, 4.1))
        .opts(cmap="bjy", colorbar=True, clabel="AOT at 10um")
    )

In [ ]:
hv.Layout(layout_pred).cols(1)